<h2>Implemented from</h2>
<h3><a href="https://github.com/silencedsre/Attention/blob/master/NMT.ipynb"> Resource</a></h3>

In [5]:
import tensorflow as tf

In [10]:
tf.__version__

'2.0.0'

In [17]:
dense_units = 5
vocab_size = 100
embedding_dim = 3
# the length determines the no. of time step or hidden state
max_length = 6

gru_units = 10
batch_size = 3

In [18]:
inputs = tf.Variable([
                [1, 3, 4, 10, 54, 2], # eg: <start> this is a  bag <end>
                [1, 30, 4, 10, 76, 2],   #eg: <start> he is a boy <end>
                [1, 3, 4, 10, 34, 2] # eg: <start> this is a bird <end>
                     ])
print(f"Inputs: \n{inputs.numpy()}")

Inputs: 
[[ 1  3  4 10 54  2]
 [ 1 30  4 10 76  2]
 [ 1  3  4 10 34  2]]


In [19]:
#Input Shape
inputs.shape

TensorShape([3, 6])

In [20]:
enc_embedding = tf.keras.layers.Embedding(vocab_size,embedding_dim)
x = enc_embedding(inputs)

In [22]:
x.numpy()

array([[[-0.002189  ,  0.03387812,  0.03366368],
        [-0.02527224, -0.0253464 , -0.04970657],
        [ 0.03916557,  0.0229264 ,  0.02924533],
        [-0.02816937, -0.02453755,  0.02061386],
        [-0.0455017 , -0.01238099, -0.0237707 ],
        [-0.04621479, -0.01939863, -0.03276088]],

       [[-0.002189  ,  0.03387812,  0.03366368],
        [-0.02060975,  0.01104768,  0.03671408],
        [ 0.03916557,  0.0229264 ,  0.02924533],
        [-0.02816937, -0.02453755,  0.02061386],
        [-0.03920903, -0.02873352,  0.00734175],
        [-0.04621479, -0.01939863, -0.03276088]],

       [[-0.002189  ,  0.03387812,  0.03366368],
        [-0.02527224, -0.0253464 , -0.04970657],
        [ 0.03916557,  0.0229264 ,  0.02924533],
        [-0.02816937, -0.02453755,  0.02061386],
        [-0.04598507,  0.01879999, -0.03811921],
        [-0.04621479, -0.01939863, -0.03276088]]], dtype=float32)

In [30]:
# batch_size, max_length, embedding_dims
x.shape

TensorShape([3, 6, 3])

In [40]:
# Using GRU units on the encoder side
enc_gru = tf.keras.layers.GRU(gru_units,
                               return_sequences=True,
                               return_state=True,
                               recurrent_initializer='glorot_uniform')

In [43]:
# enc_hidden is allocated value only when return_state is set to True
# similarly return_sequences serve the neighbouring hidden states 
enc_output, enc_hidden = enc_gru(x)

In [52]:
print("Encoder Output = " + str(enc_output.shape))
print("Hidden State Output = " + str(enc_hidden.shape))

Encoder Output = (3, 6, 10)
Hidden State Output = (3, 10)


In [53]:
values = enc_output

In [54]:
## Now Lets go through the Queries

In [55]:
query = enc_hidden
print(query.shape)

(3, 10)


In [60]:
#So, the query with time axis becomes
query_with_time_axis = tf.expand_dims(query, 1)
print("Query with time layer:")
query_with_time_axis.shape

Query with time layer:


TensorShape([3, 1, 10])

In [63]:
# Attention Layers
query_layer = tf.keras.layers.Dense(dense_units)
value_layer = tf.keras.layers.Dense(dense_units)
V = tf.keras.layers.Dense(1)